<a href="https://colab.research.google.com/github/darshan12x/projects/blob/mainbranch/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tweepy
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import numpy as np 
import matplotlib.pyplot as plt 
import string
import nltk
import warnings
import seaborn as sns
from textblob import TextBlob
warnings.filterwarnings("ignore", category=DeprecationWarning)
nltk.download('vader_lexicon')

%matplotlib inline

In [0]:
#My Twitter API Authentication Variables
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [6]:
tweets = api.search('China', count=200) # search term 

# 'iso_language_code': 'en'
# 'location': ''
# 'followers_count': 41
# 'verified': False
# 'retweeted': False

data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

display(data.head(10))

,Tweets
0,Mitt Romney: America is awakening to China. Th...
1,RT @billieeilish: Billie on the cover of Vogue...
2,RT @TsangMali: @FT Hoping that #COVID19 can be...
3,RT @RepAGonzalez: Glad to see @CommerceGov iss...
4,@SkyNews He’s demented and I have the feeling ...
5,RT @TaiwanNews886: China threatens to halt med...
6,RT @itsJeffTiedrich: 15 cases didn't go down t...
7,RT @RawStory: New information comes to light o...
8,@XHNews Damage control. China is only trying t...
9,@pepper46477330 @China_Welt_News Nicht?


In [7]:
# tidy up tweets
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

# remove twitter handles (@user)
data['tidy_tweet'] = np.vectorize(remove_pattern)(data['Tweets'], "@[\w]*")

# remove links
data['tidy_tweet'] = np.vectorize(remove_pattern)(data['tidy_tweet'],"(\w+:\/\/\S+)")

# for hashtags - "#(\w+)" 

# remove special characters, numbers, punctuations
data['tidy_tweet'] = data['tidy_tweet'].str.replace("([^0-9A-Za-z \t])", " ")

# removing short words
data['tidy_tweet'] = data['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

display(data.head(10))

,Tweets,tidy_tweet
0,Mitt Romney: America is awakening to China. Th...,Mitt Romney America awakening China This clari...
1,RT @billieeilish: Billie on the cover of Vogue...,Billie the cover Vogue China
2,RT @TsangMali: @FT Hoping that #COVID19 can be...,Hoping that COVID19 can Chernobyl China CCP am...
3,RT @RepAGonzalez: Glad to see @CommerceGov iss...,Glad see issue new trade regulations China hel...
4,@SkyNews He’s demented and I have the feeling ...,demented and have the feeling were lose the el...
5,RT @TaiwanNews886: China threatens to halt med...,China threatens halt medical supplies after Ne...
6,RT @itsJeffTiedrich: 15 cases didn't go down t...,cases didn down didn disappear like miracle th...
7,RT @RawStory: New information comes to light o...,New information comes light Trump business dea...
8,@XHNews Damage control. China is only trying t...,Damage control China only trying save face
9,@pepper46477330 @China_Welt_News Nicht?,Nicht


In [0]:
# Alternate func
# regex to strips URL, punctuations, usernames and any non-alphanumeric chars. Also separates words with single space. 
""" 
def clean_tweet(tweet):
  return ' '.join(re.sub("@[\w]*|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())

data['tidy_tweet'] = pd.DataFrame(data=[clean_tweet(tweet.text) for tweet in tweets], columns=['Tweets'])
data['tidy_tweet'] = data['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
display(data.head(10))

"""

In [8]:
sid = SentimentIntensityAnalyzer()
listy = []

for index, row in data.iterrows():
  ss = sid.polarity_scores(row["tidy_tweet"])
  listy.append(ss)
  
se = pd.Series(listy)
data['polarity'] = se.values

display(data.head(100))

,Tweets,tidy_tweet,polarity
0,Mitt Romney: America is awakening to China. Th...,Mitt Romney America awakening China This clari...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,RT @billieeilish: Billie on the cover of Vogue...,Billie the cover Vogue China,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,RT @TsangMali: @FT Hoping that #COVID19 can be...,Hoping that COVID19 can Chernobyl China CCP am...,"{'neg': 0.229, 'neu': 0.649, 'pos': 0.121, 'co..."
3,RT @RepAGonzalez: Glad to see @CommerceGov iss...,Glad see issue new trade regulations China hel...,"{'neg': 0.0, 'neu': 0.695, 'pos': 0.305, 'comp..."
4,@SkyNews He’s demented and I have the feeling ...,demented and have the feeling were lose the el...,"{'neg': 0.174, 'neu': 0.645, 'pos': 0.181, 'co..."
...,...,...,...
95,RT @gustavoforgione: Presidencia obligó la lib...,Presidencia oblig liberaci del pesquero rong Y...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
96,RT @mel_wright123: @ElaineM11584892 @EICosthet...,what was all the ranting about transparency Ch...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
97,RT @johnpilger: There is a coming war on China...,There coming war China The tip the spear Austr...,"{'neg': 0.326, 'neu': 0.608, 'pos': 0.066, 'co..."
98,#FotoProfilBaru waktu Asian Games 2018 vs Chin...,FotoProfilBaru waktu Asian Games 2018 China Ta...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [9]:
#  tokenize all the cleaned tweets in our dataset
tokenized_tweet = data['tidy_tweet'].apply(lambda x: x.split())

tokenized_tweet.head()

0    [Mitt, Romney, America, awakening, China, This...
1                   [Billie, the, cover, Vogue, China]
2    [Hoping, that, COVID19, can, Chernobyl, China,...
3    [Glad, see, issue, new, trade, regulations, Ch...
4    [demented, and, have, the, feeling, were, lose...
Name: tidy_tweet, dtype: object